# Description

Выполнение ДЗ:

Для выполнения задания (добавление 1 FileSource и двух on_demand_feature_view) я сделал:

1) новый источник (еще один паркет) с агрегированными данными. Положил его в туже папку.
В паркетнике посчитаны все средние значения по трем фичам (отдельно для кажлого driver_id)

2) далее сделал еще один FileSource уже из этих агегированных данных

3) посчитал 2 on demand группы фичей. Одна производит нормолизацию данных по трем фичам ( (X-mean)/ std)

4) Вторая делит текущиее значение, на среднее по данному driver_id. Получаем относительную фичу, которая показывает выше/ниже оно для данного driver_id
и насколько




# Import

In [ ]:
import os
from datetime import datetime

import pandas as pd
from feast import FeatureStore

In [2]:
!pwd

/home/ubuntu/tidy_ghost


In [3]:
raw_data_path = os.path.join("feature_repo","data","driver_stats.parquet")

feature_store_path = '/home/ubuntu/tidy_ghost/feature_repo'

# загрузка данных

In [5]:
df = pd.read_parquet(raw_data_path)
df.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2024-11-06 15:00:00+00:00,1005,0.652581,0.175324,743,2024-11-21 15:36:55.587
1,2024-11-06 16:00:00+00:00,1005,0.978495,0.098539,177,2024-11-21 15:36:55.587
2,2024-11-06 17:00:00+00:00,1005,0.431618,0.620169,106,2024-11-21 15:36:55.587
3,2024-11-06 18:00:00+00:00,1005,0.266548,0.401481,68,2024-11-21 15:36:55.587
4,2024-11-06 19:00:00+00:00,1005,0.606263,0.668137,812,2024-11-21 15:36:55.587


In [41]:
df.dtypes

event_timestamp    datetime64[ns, UTC]
driver_id                        int64
conv_rate                      float32
acc_rate                       float32
avg_daily_trips                  int32
created                 datetime64[us]
dtype: object

## Feature inference

In [7]:
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001,1002,1003],
        "event_timestamp": [
            datetime(2021,4,12,10,59,42),
            datetime(2021,4,12,8,12,10),
            datetime(2021,4,12,16,40,26),

        ],
        "label_driver_reported_satisfaction": [1,5,3]


    }
)

In [8]:
entity_df

,driver_id,event_timestamp,label_driver_reported_satisfaction
0,1001,2021-04-12 10:59:42,1
1,1002,2021-04-12 08:12:10,5
2,1003,2021-04-12 16:40:26,3


In [28]:
store = FeatureStore(repo_path = feature_store_path)

In [12]:
training_df = store.get_historical_features(
    entity_df = entity_df,
    features = [
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
        ],
).to_df()

training_df.head()

,driver_id,event_timestamp,label_driver_reported_satisfaction,conv_rate,acc_rate,avg_daily_trips
0,1001,2021-04-12 10:59:42+00:00,1,0.384781,0.490758,962
1,1002,2021-04-12 08:12:10+00:00,5,0.710138,0.909994,567
2,1003,2021-04-12 16:40:26+00:00,3,0.655839,0.873373,221


## Feature inference on demand

In [13]:
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001,1002,1003],
        "event_timestamp": [
            datetime(2021,4,12,10,59,42),
            datetime(2021,4,12,8,12,10),
            datetime(2021,4,12,16,40,26),

        ],
        "label_driver_reported_satisfaction": [1,5,3],
        "val_to_add": [1,2,3],
        "val_to_add_2": [10,20,30]


    }
)

entity_df

,driver_id,event_timestamp,label_driver_reported_satisfaction,val_to_add,val_to_add_2
0,1001,2021-04-12 10:59:42,1,1,10
1,1002,2021-04-12 08:12:10,5,2,20
2,1003,2021-04-12 16:40:26,3,3,30


In [14]:
training_df = store.get_historical_features(
    entity_df = entity_df,
    features = [
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
        "transformed_conv_rate:conv_rate_plus_val1",
        "transformed_conv_rate:conv_rate_plus_val2",
        ],
).to_df()

training_df.head()

,driver_id,event_timestamp,label_driver_reported_satisfaction,val_to_add,val_to_add_2,conv_rate,acc_rate,avg_daily_trips,conv_rate_plus_val1,conv_rate_plus_val2
0,1001,2021-04-12 10:59:42+00:00,1,1,10,0.384781,0.490758,962,1.384781,10.384781
1,1002,2021-04-12 08:12:10+00:00,5,2,20,0.710138,0.909994,567,2.710138,20.710138
2,1003,2021-04-12 16:40:26+00:00,3,3,30,0.655839,0.873373,221,3.655839,30.655839


## Расчет нового источника (агрегации)

In [12]:
df = pd.read_parquet(raw_data_path)

df_test = df.copy()

In [13]:
df_test.columns

cols = ['conv_rate', 'acc_rate',
       'avg_daily_trips']

aggs = ['mean','min','max']

In [14]:
df_test.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2024-11-06 15:00:00+00:00,1005,0.652581,0.175324,743,2024-11-21 15:36:55.587
1,2024-11-06 16:00:00+00:00,1005,0.978495,0.098539,177,2024-11-21 15:36:55.587
2,2024-11-06 17:00:00+00:00,1005,0.431618,0.620169,106,2024-11-21 15:36:55.587
3,2024-11-06 18:00:00+00:00,1005,0.266548,0.401481,68,2024-11-21 15:36:55.587
4,2024-11-06 19:00:00+00:00,1005,0.606263,0.668137,812,2024-11-21 15:36:55.587


In [15]:
df_test.columns

Index(['event_timestamp', 'driver_id', 'conv_rate', 'acc_rate',
       'avg_daily_trips', 'created'],
      dtype='object')

In [16]:
cols_final = ['event_timestamp','driver_id']
#df_test['gefault_time']=datetime(2099,1,1,0,0,0)
for col in cols:
    for agg in aggs:
        df_test[f'{col}_{agg}']=df_test.groupby(['driver_id'])[col].transform(agg)
        cols_final.append(f'{col}_{agg}')

df_test=df_test[cols_final]#.drop_duplicates()

df_test['created']=datetime(2024,11,21,0,0,0)

df_test.head()   



,event_timestamp,driver_id,conv_rate_mean,conv_rate_min,conv_rate_max,acc_rate_mean,acc_rate_min,acc_rate_max,avg_daily_trips_mean,avg_daily_trips_min,avg_daily_trips_max,created
0,2024-11-06 15:00:00+00:00,1005,0.506842,0.000154,0.999657,0.508275,0.001269,0.997274,497.65374,3,982,2024-11-21
1,2024-11-06 16:00:00+00:00,1005,0.506842,0.000154,0.999657,0.508275,0.001269,0.997274,497.65374,3,982,2024-11-21
2,2024-11-06 17:00:00+00:00,1005,0.506842,0.000154,0.999657,0.508275,0.001269,0.997274,497.65374,3,982,2024-11-21
3,2024-11-06 18:00:00+00:00,1005,0.506842,0.000154,0.999657,0.508275,0.001269,0.997274,497.65374,3,982,2024-11-21
4,2024-11-06 19:00:00+00:00,1005,0.506842,0.000154,0.999657,0.508275,0.001269,0.997274,497.65374,3,982,2024-11-21


In [15]:
df_test.dtypes

driver_id                        int64
conv_rate_mean                 float32
conv_rate_min                  float32
conv_rate_max                  float32
acc_rate_mean                  float32
acc_rate_min                   float32
acc_rate_max                   float32
avg_daily_trips_mean           float64
avg_daily_trips_min              int32
avg_daily_trips_max              int32
event_timestamp_def     datetime64[us]
dtype: object

In [39]:
cols_final

['event_timestamp',
 'driver_id',
 'conv_rate_mean',
 'conv_rate_min',
 'conv_rate_max',
 'acc_rate_mean',
 'acc_rate_min',
 'acc_rate_max',
 'avg_daily_trips_mean',
 'avg_daily_trips_min',
 'avg_daily_trips_max']

In [17]:


upload_data_path = os.path.join("feature_repo","data","driver_stats_agg.parquet")

df_test.to_parquet(upload_data_path)

## Тестирование добавления новых фичей

In [40]:
store = FeatureStore(repo_path = feature_store_path)

In [43]:
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001,1002,1003],
        "event_timestamp": [
            datetime(2021,4,12,10,59,42),
            datetime(2021,4,12,8,12,10),
            datetime(2021,4,12,16,40,26),

        ],

        # "label_driver_reported_satisfaction": [1,5,3],
        # "val_to_add": [1,2,3],
        # "val_to_add_2": [10,20,30]
    }
)

entity_df

,driver_id,event_timestamp
0,1001,2021-04-12 10:59:42
1,1002,2021-04-12 08:12:10
2,1003,2021-04-12 16:40:26


In [45]:
training_df = store.get_historical_features(
    entity_df = entity_df,
    features = [
        #базовые фичи
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
        #средние значения
        "driver_hourly_stats_agg:conv_rate_mean",
        "driver_hourly_stats_agg:acc_rate_mean",
        "driver_hourly_stats_agg:avg_daily_trips_mean",
        # отношение к среднему по driver_id
        "transformed_feature_rel:conv_rate_rel_to_mean",
        "transformed_feature_rel:acc_rate_to_mean",
        "transformed_feature_rel:avg_daily_trips_to_mean",
        # нормализованные фичи
        "transformed_feature_norm:conv_rate_norm",
        "transformed_feature_norm:acc_rate_norm",
        "transformed_feature_norm:avg_daily_trips_norm",

        ],
).to_df()

training_df.head()

,driver_id,event_timestamp,conv_rate,acc_rate,avg_daily_trips,conv_rate_mean,acc_rate_mean,avg_daily_trips_mean,conv_rate_norm,acc_rate_norm,avg_daily_trips_norm,conv_rate_rel_to_mean,acc_rate_to_mean,avg_daily_trips_to_mean
0,1001,2021-04-12 10:59:42+00:00,0.384781,0.490758,962,0.510318,0.509142,498.648199,-1.140606,-1.151105,1.021298,0.754002,0.963892,1.929216
1,1002,2021-04-12 08:12:10+00:00,0.710138,0.909994,567,0.512824,0.493920,522.304709,0.726070,0.654410,-0.044052,1.384761,1.842392,1.085573
2,1003,2021-04-12 16:40:26+00:00,0.655839,0.873373,221,0.481869,0.498969,471.504132,0.414537,0.496695,-0.977246,1.361032,1.750356,0.468713


# Выводы

Добавлены 3 новых группы фичей. 2 из on demand, и одна из FeatureView